In [ ]:
# Dependências

In [1]:
%pip install pyspark
%pip install pandas
%pip install xlrd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Funções de cálculo 

In [78]:
import math

def dms_para_dd(graus, minutos, segundos, direcao):
    # Converter para graus decimais
    decimal = graus + (minutos / 60) + (segundos / 3600)
    
    # Ajustar o sinal para Sul e Oeste
    if direcao in ['S', 'W']:
        decimal = -decimal
    
    return decimal

In [3]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # Raio da Terra em km
    R = 6371.0
    
    # Converter de graus para radianos
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    # Diferença de coordenadas
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Distância
    distance = R * c
    return distance

In [4]:
from pyspark.sql import functions as F
haversine_udf = F.udf(haversine)

In [ ]:
# Spark session

In [5]:
from pyspark.sql import SparkSession
import pandas as pd
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .appName("Analytics e BI") \
    .getOrCreate()

24/10/09 18:53:18 WARN Utils: Your hostname, Ivans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.181 instead (on interface en0)
24/10/09 18:53:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/09 18:53:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
pd_recl1 = pd.read_csv('./dadosconsumidor2022.csv', skiprows=1, sep=';')
reclamacoes_1 = spark.createDataFrame(pd_recl1)

In [7]:
pd_recl2= pd.read_csv('./dadosconsumidor2023.csv', skiprows=1, sep=';')
reclamacoes_2 = spark.createDataFrame(pd_recl2)

In [9]:
# Junta as tabelas em uma só

In [8]:
reclamacoes = reclamacoes_1.union(reclamacoes_2)

In [9]:
aeroportos = spark.createDataFrame(pd.read_excel('./aeroportos.xls', skiprows=2))

In [10]:
empresas_aereas = spark.createDataFrame(pd.read_excel('./empresas_aereas.xls', skiprows=3))\
                        .drop('Unnamed: 0','Unnamed: 3')

In [11]:
voos = spark.read.csv('./VRA_20240916110513.csv', sep =';' ,header=True)

In [39]:
print(f"Voos: {voos}\n")
print(f"empresas_aereas: {empresas_aereas}\n")
print(f"aeroportos: {aeroportos}\n")
print(f"reclamacoes: {reclamacoes}\n")

Voos: DataFrame[Sigla ICAO Empresa Aérea: string, Empresa Aérea: string, Número Voo: string, Código DI: string, Código Tipo Linha: string, Modelo Equipamento: string, Número de Assentos: string, Sigla ICAO Aeroporto Origem: string, Descrição Aeroporto Origem: string, Partida Prevista: string, Partida Real: string, Sigla ICAO Aeroporto Destino: string, Descrição Aeroporto Destino: string, Chegada Prevista: string, Chegada Real: string, Situação Voo: string, Justificativa: string, Referência: string, Situação Partida: string, Situação Chegada: string]

empresas_aereas: DataFrame[Sigla OACI: string, Nome Empresas: string, Nacional ou Estrangeira: string]

aeroportos: DataFrame[CÓDIGO OACI: string, CIAD: string, NOME: string, MUNICÍPIO ATENDIDO: string, UF: string, LATITUDE: string, LONGITUDE: string, ALTITUDE: string, OPERAÇÃO: string, DESIGNAÇÃO: string, COMPRIMENTO: string, LARGURA: string, RESISTÊNCIA: string, SUPERFÍCIE: string, DESIGNAÇÃO.1: string, COMPRIMENTO.1: string, LARGURA.1: 

In [36]:
# Pega apenas voos domésticos

In [46]:
voos_domesticos = voos.filter(
    (F.col('Descrição Aeroporto Origem').contains('BRASIL'))
    &
    (F.col('Descrição Aeroporto Destino').contains('BRASIL'))
)

In [41]:
# Conta voos domésticos

In [13]:
voos_domesticos.count()

1304313

In [43]:
# Tratando a base de aeroportos

In [14]:
aeroportos.printSchema()

root
 |-- CÓDIGO OACI: string (nullable = true)
 |-- CIAD: string (nullable = true)
 |-- NOME: string (nullable = true)
 |-- MUNICÍPIO ATENDIDO: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- ALTITUDE: string (nullable = true)
 |-- OPERAÇÃO: string (nullable = true)
 |-- DESIGNAÇÃO: string (nullable = true)
 |-- COMPRIMENTO: string (nullable = true)
 |-- LARGURA: string (nullable = true)
 |-- RESISTÊNCIA: string (nullable = true)
 |-- SUPERFÍCIE: string (nullable = true)
 |-- DESIGNAÇÃO.1: string (nullable = true)
 |-- COMPRIMENTO.1: string (nullable = true)
 |-- LARGURA.1: string (nullable = true)
 |-- RESISTÊNCIA.1: string (nullable = true)
 |-- SUPERFÍCIE.1: string (nullable = true)
 |-- DESIGNAÇÃO.2: string (nullable = true)
 |-- COMPRIMENTO.2: string (nullable = true)
 |-- LARGURA.2: string (nullable = true)
 |-- RESISTÊNCIA.2: string (nullable = true)
 |-- SUPERFÍCIE.2: string (nulla

In [47]:
aeroportos.withColumn('split_lat', F.split('LATITUDE',' '))\
          .withColumn('split_long', F.split('LONGITUDE', ' '))\
          .show(1, False)

+-----------+------+---------------------+---------------------+---+-------------+--------------+--------+-------------------------------+----------+-----------+-------+--------------+----------+------------+-------------+---------+-------------+------------+------------+-------------+---------+-------------+------------+-----------------------+------------------------+---------+-------------+------------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+---------------------------------+------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------+----------------+---------------------------+-------------+---+---+-------------+--------------+--------+----+----------+-------------------------+-----------+------------------+-------------------+
|CÓDIGO OACI|CIAD  |NOME                 |MUNICÍPIO ATENDIDO   |UF |LATITUDE     |LONGITUDE     |ALTITUDE|OPE

In [48]:
colunas_df_aeroportos = ['CÓDIGO OACI','LATITUDE', 'LONGITUDE']
aeroportos_origem = aeroportos.select(colunas_df_aeroportos)
aeroportos_dest = aeroportos.select(colunas_df_aeroportos)

for coluna in aeroportos.columns:
  aeroportos_origem = aeroportos_origem.withColumnRenamed(coluna, f'{coluna}_orig')

for coluna in aeroportos.columns:
  aeroportos_dest = aeroportos_dest.withColumnRenamed(coluna, f'{coluna}_dest')

In [49]:
print("Aeroporto dest")
aeroportos_dest.printSchema()
print("Aeroporto Origem")
aeroportos_origem.printSchema()

Aeroporto dest
root
 |-- CÓDIGO OACI_dest: string (nullable = true)
 |-- LATITUDE_dest: string (nullable = true)
 |-- LONGITUDE_dest: string (nullable = true)

Aeroporto Origem
root
 |-- CÓDIGO OACI_orig: string (nullable = true)
 |-- LATITUDE_orig: string (nullable = true)
 |-- LONGITUDE_orig: string (nullable = true)



In [50]:
aeroportos_origem.show(1, False)
aeroportos_dest.show(1, True)

+----------------+-------------+--------------+
|CÓDIGO OACI_orig|LATITUDE_orig|LONGITUDE_orig|
+----------------+-------------+--------------+
|SBAA            |8° 20' 55'' S|49° 18' 11'' W|
+----------------+-------------+--------------+
only showing top 1 row

+----------------+-------------+--------------+
|CÓDIGO OACI_dest|LATITUDE_dest|LONGITUDE_dest|
+----------------+-------------+--------------+
|            SBAA|8° 20' 55'' S|49° 18' 11'' W|
+----------------+-------------+--------------+
only showing top 1 row



In [51]:
aeroportos_voo_orig = voos.join(aeroportos_origem, (F.col('Sigla ICAO Aeroporto Origem') == F.col('CÓDIGO OACI_orig')))

aeroportos_voo_orig_dest = aeroportos_voo_orig.join(aeroportos_dest, (F.col('Sigla ICAO Aeroporto Origem') == F.col('CÓDIGO OACI_dest')), 'left')

In [83]:
aeroportos_voo_orig_dest.show(1, False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------
 Sigla ICAO Empresa Aérea     | ACN                                                   
 Empresa Aérea                | AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA)           
 Número Voo                   | 5991                                                  
 Código DI                    | 6                                                     
 Código Tipo Linha            | N                                                     
 Modelo Equipamento           | C208                                                  
 Número de Assentos           | 0                                                     
 Sigla ICAO Aeroporto Origem  | SBPS                                                  
 Descrição Aeroporto Origem   | PORTO SEGURO - PORTO SEGURO - BA - BRASIL             
 Partida Prevista             | NULL                                                  
 Partida Real                 | 28/01/2023 

In [76]:
from pyspark.sql.types import FloatType
dms_para_dd_udf_spark = F.udf(dms_para_dd_udf, FloatType())

In [89]:
def extrair_dms(dms_str):
    match = re.match(r"(\d+)° (\d+)' (\d+)'' ([NSWE])", dms_str)
    if match:
        graus = int(match.group(1))
        minutos = int(match.group(2))
        segundos = int(match.group(3))
        direcao = match.group(4)
        return (graus, minutos, segundos, direcao)
    return (None, None, None, None)

# Função UDF que aplica a conversão
def dms_converter_udf(dms_str):
    graus, minutos, segundos, direcao = extrair_dms(dms_str)
    if graus is not None:
        return dms_para_dd(graus, minutos, segundos, direcao)
    return None

In [90]:
dms_udf = F.udf(dms_converter_udf, FloatType())

# Aplicar a função no DataFrame para converter as colunas
df = aeroportos_voo_orig_dest \
    .withColumn("LAT_DEC_orig", dms_udf(F.col("LATITUDE_orig"))) \
    .withColumn("LONG_DEC_orig", dms_udf(F.col("LONGITUDE_orig"))) \
    .withColumn("LAT_DEC_dest", dms_udf(F.col("LATITUDE_dest"))) \
    .withColumn("LONG_DEC_dest", dms_udf(F.col("LONGITUDE_dest")))

In [92]:
df_with_distance = df.withColumn('Distância (km)', haversine_udf(
    F.col('LAT_DEC_orig'), 
    F.col('LONG_DEC_orig'), 
    F.col('LAT_DEC_dest'), 
    F.col('LONG_DEC_dest')
))

In [94]:
df_with_distance.show(10, False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------
 Sigla ICAO Empresa Aérea     | ACN                                                   
 Empresa Aérea                | AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA)           
 Número Voo                   | 5991                                                  
 Código DI                    | 6                                                     
 Código Tipo Linha            | N                                                     
 Modelo Equipamento           | C208                                                  
 Número de Assentos           | 0                                                     
 Sigla ICAO Aeroporto Origem  | SBPS                                                  
 Descrição Aeroporto Origem   | PORTO SEGURO - PORTO SEGURO - BA - BRASIL             
 Partida Prevista             | NULL                                                  
 Partida Real                 | 28/01/2023 

In [44]:
colunas_df_aeroportos = ['CÓDIGO OACI', 'LATITUDE', 'LONGITUDE']
aeroportos_origem_fmt = aeroportos.select(colunas_df_aeroportos).withColumnRenamed('CÓDIGO OACI', 'CÓDIGO OACI_orig')
aeroportos_dest_fmt = aeroportos.select(colunas_df_aeroportos).withColumnRenamed('CÓDIGO OACI', 'CÓDIGO OACI_dest')


In [33]:
aeroportos_origem_fmt.show(1, False, vertical=True)

-RECORD 0--------------------------
 CÓDIGO OACI_orig | SBAA           
 LATITUDE         | 8° 20' 55'' S  
 LONGITUDE        | 49° 18' 11'' W 
only showing top 1 row



In [55]:
aeroportos_voo_orig_join_aoci = voos.join(aeroportos_origem, (F.col('Sigla ICAO Aeroporto Origem') == F.col('CÓDIGO OACI_orig')))
aeroportos_voo_orig_dest_aoci = aeroportos_voo_orig.join(aeroportos_dest, (F.col('Sigla ICAO Aeroporto Destino') == F.col('CÓDIGO OACI_dest')))


In [56]:
aeroportos_voo_orig_join_aoci.show(1, False, vertical=True)
#aeroportos_voo_orig_dest_aoci.show(1, False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------
 Sigla ICAO Empresa Aérea     | ACN                                                   
 Empresa Aérea                | AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA)           
 Número Voo                   | 5991                                                  
 Código DI                    | 6                                                     
 Código Tipo Linha            | N                                                     
 Modelo Equipamento           | C208                                                  
 Número de Assentos           | 0                                                     
 Sigla ICAO Aeroporto Origem  | SBPS                                                  
 Descrição Aeroporto Origem   | PORTO SEGURO - PORTO SEGURO - BA - BRASIL             
 Partida Prevista             | NULL                                                  
 Partida Real                 | 28/01/2023 

In [70]:
voos_domesticos.printSchema()

root
 |-- Sigla ICAO Empresa Aérea: string (nullable = true)
 |-- Empresa Aérea: string (nullable = true)
 |-- Número Voo: string (nullable = true)
 |-- Código DI: string (nullable = true)
 |-- Código Tipo Linha: string (nullable = true)
 |-- Modelo Equipamento: string (nullable = true)
 |-- Número de Assentos: string (nullable = true)
 |-- Sigla ICAO Aeroporto Origem: string (nullable = true)
 |-- Descrição Aeroporto Origem: string (nullable = true)
 |-- Partida Prevista: string (nullable = true)
 |-- Partida Real: string (nullable = true)
 |-- Sigla ICAO Aeroporto Destino: string (nullable = true)
 |-- Descrição Aeroporto Destino: string (nullable = true)
 |-- Chegada Prevista: string (nullable = true)
 |-- Chegada Real: string (nullable = true)
 |-- Situação Voo: string (nullable = true)
 |-- Justificativa: string (nullable = true)
 |-- Referência: string (nullable = true)
 |-- Situação Partida: string (nullable = true)
 |-- Situação Chegada: string (nullable = true)
 |-- CÓDIGO OAC

In [57]:
voos_domesticos.show(1, False, vertical=True)

-RECORD 0------------------------------------------------------------------------
 Sigla ICAO Empresa Aérea     | 1ED                                              
 Empresa Aérea                | SERVICIOS AÉREOS PANAMERICANOS LTDA. SARPA S.A.S 
 Número Voo                   | 5330                                             
 Código DI                    | 7                                                
 Código Tipo Linha            | X                                                
 Modelo Equipamento           | E145                                             
 Número de Assentos           | 50                                               
 Sigla ICAO Aeroporto Origem  | SBCY                                             
 Descrição Aeroporto Origem   | MARECHAL RONDON - VÁRZEA GRANDE - MT - BRASIL    
 Partida Prevista             | NULL                                             
 Partida Real                 | 31/08/2023 07:07                                 
 Sigla ICAO Aero

In [ ]:
# Join na longitude latitude destino

In [42]:
#voos_domesticos_aux = voos_domesticos.drop('LONGITUDE_dest')
#voos_domesticos_aux = voos_domesticos_aux.drop('LATITUDE_dest')
#voos_domesticos_aux = voos_domesticos_aux.drop('LATITUDE_orig')
#voos_domesticos_aux = voos_domesticos_aux.drop('LONGITUDE_orig')


voos_atualizados_dest = voos_domesticos_aux.join(
    aeroportos_voo_orig_dest_aoci.select(
        col("Sigla ICAO Empresa Aérea"),
        col("Número Voo"),
        col("CÓDIGO OACI_dest").alias("CÓDIGO OACI_dest"),
        col("LATITUDE_dest"),
        col("LONGITUDE_dest")
    ),
    (voos_domesticos["Sigla ICAO Empresa Aérea"] == aeroportos_voo_orig_dest_aoci["Sigla ICAO Empresa Aérea"]) &
    (voos_domesticos["Número Voo"] == aeroportos_voo_orig_dest_aoci["Número Voo"]) &
    (voos_domesticos["CÓDIGO OACI_dest"] == aeroportos_voo_orig_dest_aoci["CÓDIGO OACI_dest"]),
    "left"
)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `CÓDIGO OACI_dest` cannot be resolved. Did you mean one of the following? [`Sigla ICAO Empresa Aérea`, `Empresa Aérea`, `Número Voo`, `Código DI`, `Código Tipo Linha`, `Modelo Equipamento`, `Número de Assentos`, `Sigla ICAO Aeroporto Origem`, `Descrição Aeroporto Origem`, `Partida Prevista`, `Partida Real`, `Sigla ICAO Aeroporto Destino`, `Descrição Aeroporto Destino`, `Chegada Prevista`, `Chegada Real`, `Situação Voo`, `Justificativa`, `Referência`, `Situação Partida`, `Situação Chegada`].

In [38]:
voos_atualizados_orig = voos_domesticos_aux.join(
    aeroportos_voo_orig_join_aoci.select(
        col("Sigla ICAO Empresa Aérea"),
        col("Número Voo"),
        col("CÓDIGO OACI_orig").alias("CÓDIGO OACI_orig"),
        col("LATITUDE_orig"),
        col("LONGITUDE_orig")
    ),
    (voos_domesticos["Sigla ICAO Empresa Aérea"] == aeroportos_voo_orig_dest_aoci["Sigla ICAO Empresa Aérea"]) &
    (voos_domesticos["Número Voo"] == aeroportos_voo_orig_dest_aoci["Número Voo"]) &
    (voos_domesticos["CÓDIGO OACI_orig"] == aeroportos_voo_orig_dest_aoci["CÓDIGO OACI_orig"]),
    "left"
)

NameError: name 'col' is not defined

In [127]:
voos_combined = voos_atualizados_orig.join(
    voos_atualizados_dest,
    on=["Sigla ICAO Empresa Aérea", "Número Voo"],
    how="left"
)

In [39]:
from pyspark.sql.functions import col
voos_domesticos = voos_combined \
    .withColumn('LATITUDE_orig', col('LATITUDE_orig').cast('float')) \
    .withColumn('LONGITUDE_orig', col('LONGITUDE_orig').cast('float')) \
    .withColumn('LATITUDE_dest', col('LATITUDE_dest').cast('float')) \
    .withColumn('LONGITUDE_dest', col('LONGITUDE_dest').cast('float'))

NameError: name 'voos_combined' is not defined

# 3
Na base disponibilizada não temos os assentos ocupados,
somente os assentos totais das aeronaves, de forma
randômica criem a informação de assentos ocupados sendo
que a logica deve obedecer ao critério de no mínimo 50% de
ocupação e no máximo 95% de ocupação

In [95]:
voos.show(1, False, vertical=True)

-RECORD 0---------------------------------------------------------------------------------------
 Sigla ICAO Empresa Aérea     | 1ED                                                             
 Empresa Aérea                | SERVICIOS AÉREOS PANAMERICANOS LTDA. SARPA S.A.S                
 Número Voo                   | 5330                                                            
 Código DI                    | 7                                                               
 Código Tipo Linha            | I                                                               
 Modelo Equipamento           | E145                                                            
 Número de Assentos           | 50                                                              
 Sigla ICAO Aeroporto Origem  | SKLT                                                            
 Descrição Aeroporto Origem   | ALFREDO VÁSQUEZ COBO INTERNATIONAL AIRPORT - LETICIA - COLÔMBIA 
 Partida Prevista             

In [ ]:
from pyspark.sql import functions as F

In [97]:
min_ocupacao = 0.5  
max_ocupacao = 0.95 

voos_with_occupation = voos.withColumn(
    "assentos_preenchidos",
    F.floor(
        (min_ocupacao + (max_ocupacao - min_ocupacao) * F.rand()) * F.col("Número de Assentos")
    )
)

In [98]:
voos_with_occupation.show(1, False, vertical=True)

-RECORD 0---------------------------------------------------------------------------------------
 Sigla ICAO Empresa Aérea     | 1ED                                                             
 Empresa Aérea                | SERVICIOS AÉREOS PANAMERICANOS LTDA. SARPA S.A.S                
 Número Voo                   | 5330                                                            
 Código DI                    | 7                                                               
 Código Tipo Linha            | I                                                               
 Modelo Equipamento           | E145                                                            
 Número de Assentos           | 50                                                              
 Sigla ICAO Aeroporto Origem  | SKLT                                                            
 Descrição Aeroporto Origem   | ALFREDO VÁSQUEZ COBO INTERNATIONAL AIRPORT - LETICIA - COLÔMBIA 
 Partida Prevista             

# Exercício 4
Gerar dados de reclamações por origem dos voos para ter a
proporcionalidade de reclamações

In [101]:
reclamacoes.show(1, False, vertical=True)

24/10/09 19:29:36 WARN TaskSetManager: Stage 90 contains a task of very large size (1980 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0--------------------------------------------------------------------------------------------------------------
 Gestor                         | Secretaria Nacional do Consumidor                                                    
 Região                         | CO                                                                                   
 UF                             | DF                                                                                   
 Cidade                         | Brasília                                                                             
 Ano Abertura                   | 2022                                                                                 
 Mês Abertura                   | 1                                                                                    
 Data Abertura                  | 2022-01-01                                                                           
 Data e Hora Resposta           | 2022-0

24/10/09 19:29:40 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 90 (TID 240): Attempting to kill Python Worker


In [102]:
voos_por_origem = voos.groupBy("Sigla ICAO Aeroporto Origem").agg(F.count("*").alias("Total_Voos"))

+---------------------------+----------+
|Sigla ICAO Aeroporto Origem|Total_Voos|
+---------------------------+----------+
|                       SBPS|     12266|
|                       SNSS|       105|
|                       SBBU|         1|
|                       LFPG|      2581|
|                       OKBK|         2|
|                       SBIH|       569|
|                       SBSJ|       339|
|                       SSSC|       209|
|                       SNTO|       400|
|                       SCIE|         6|
|                       SBCH|      3909|
|                       SNCL|       685|
|                       SAZY|        14|
|                       SBCR|       355|
|                       SNIG|       159|
|                       SBUG|       720|
|                       SN6L|        71|
|                       SBSG|     14459|
|                       SWLV|         1|
|                       SBSO|       436|
+---------------------------+----------+
only showing top

In [103]:
percentual_reclamacoes = 0.10  
reclamacoes_estimadas = voos_por_origem.withColumn(
    "Total_Reclamacoes",
    (F.col("Total_Voos") * percentual_reclamacoes).cast("integer")
)

+---------------------------+----------+-----------------+
|Sigla ICAO Aeroporto Origem|Total_Voos|Total_Reclamacoes|
+---------------------------+----------+-----------------+
|                       SBPS|     12266|             1226|
|                       SNSS|       105|               10|
|                       SBBU|         1|                0|
|                       LFPG|      2581|              258|
|                       OKBK|         2|                0|
|                       SBIH|       569|               56|
|                       SBSJ|       339|               33|
|                       SSSC|       209|               20|
|                       SNTO|       400|               40|
|                       SCIE|         6|                0|
|                       SBCH|      3909|              390|
|                       SNCL|       685|               68|
|                       SAZY|        14|                1|
|                       SBCR|       355|               3

# 6
Classificar os voos conforme a situação de partida. Todas as
linhas necessitam ter uma classificação

In [108]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
def classificar_situacao(situacao):
    if situacao is None:
        return "Indefinido"
    elif "Atraso" in situacao:
        return "Atrasado"
    elif "REALIZADO" in situacao: 
        return "Realizado"
    else:
        return "Outro"


classificar_udf = udf(classificar_situacao, StringType())

In [ ]:
voos_classificados = voos_with_occupation.withColumn("Classificacao_Situacao", classificar_udf(voos_with_occupation["Situação Partida"]))
voos_classificados.select("Número Voo", "Situação Partida", "Classificacao_Situacao").show(truncate=False)